In [2]:
from preprocessing import read_data,data_with_sparse_matrix,process_data
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import gensim
import warnings
import torch
import re
import underthesea
warnings.filterwarnings('ignore')
from string import punctuation
punctuation = punctuation + '‘’“”'


In [3]:
dat1 = read_data('Dataset_For_Work\\1-VLSP2018-SA-Hotel-train (7-3-2018).txt')
dat2 = read_data('Dataset_For_Work\\2-VLSP2018-SA-Hotel-dev (7-3-2018).txt')
dat3 = read_data('Dataset_For_Work\\3-VLSP2018-SA-Hotel-test (8-3-2018).txt')

In [82]:
dat1

,review,sentiment
0,Rộng rãi KS mới nhưng rất vắng. Các dịch vụ ch...,"{HOTEL#DESIGN&FEATURES, positive}, {HOTEL#GENE..."
1,"Địa điểm thuận tiện, trong vòng bán kính 1,5km...","{LOCATION#GENERAL, positive}"
2,"Phục vụ, view đẹp, vị trí","{SERVICE#GENERAL, positive}, {HOTEL#GENERAL, p..."
3,"thuận tiện , sạch sẽ , vui vẻ hài lòng","{HOTEL#COMFORT, positive}, {HOTEL#CLEANLINESS,..."
4,Vị trí đẹp; Có quán bar view đẹp; Nhân viên th...,"{LOCATION#GENERAL, positive}, {FACILITIES#GENE..."
...,...,...
2995,Đối diện thung lũng Một kỳ nghỉ tuyệt vời ở Sa...,"{HOTEL#COMFORT, positive}, {SERVICE#GENERAL, p..."
2996,Cảm nhận của cá nhân về Sapa View: Phòng ốc đẹ...,"{ROOMS#DESIGN&FEATURES, positive}, {ROOMS#CLEA..."
2997,"Xin chào Chudu24,Xin cám ơn dịch vụ booking củ...","{ROOMS#CLEANLINESS, positive}, {ROOMS#COMFORT,..."
2998,"Đầu tiên về ưu điểm:- Phòng sạch sẽ, nội thất ...","{ROOMS#CLEANLINESS, positive}, {ROOM_AMENITIES..."


In [8]:
final_dat1,new_dat1 = data_with_sparse_matrix(dat1)
final_dat2,new_dat2 = data_with_sparse_matrix(dat2)
final_dat3,new_dat3 = data_with_sparse_matrix(dat3)

In [4]:
final_dat1.Text.values

array(['Rộng rãi KS mới nhưng rất vắng. Các dịch vụ chất lượng chưa cao và thiếu.',
       'Địa điểm thuận tiện, trong vòng bán kính 1,5km nhiều quán ăn ngon',
       'Phục vụ, view đẹp, vị trí', ...,
       'Xin chào Chudu24,Xin cám ơn dịch vụ booking của Chudu24 đã cho vợ chồng tôi có một kỳ nghỉ yên tâm, tin tưởng và thật thoải mái. ôi xin có một số đánh giá sau đây về Sapa View hotel:1. Phòng Deluxe garden view: Phòng sạch sẽ, tiện nghi, yên tĩnh, gần trung tâm và đặc biệt view rất đẹp nhìn xuống thung lũng với ruộng bậc thang mênh mông cùng với balcony rất ấn tượng cùng giỏ hoa mầu sắc. Do ở HN chỉ có điều hòa 2 chiều nên khi ở đó mình được trải nhiệm với lò sưởi bằng củi rất đặc trưng Sapa. Buổi tối ngồi bàn đôi cạnh lò sưởi cùng ánh nến ấm áp và li rượu vang thật là tuyệt vời ông mặt zời. 2. Phục vụ: Nhân viên phục vụ là mấy em người Mông nên rất là thật thà, chu đáo. Theo quy định 2h mới được nhận phòng nhưng do đến SaPa lúc 8AM bà chủ Sapa View đã chào đón niềm nở và cho nhận 

In [ ]:
#clean_dat1 = pd.read_csv(clean_dat1.csv)

In [10]:
def process_data(data):
    sentences = []
    stop_words = open('Dataset_For_Work\\vietnamese-stopwords.txt','r',encoding='utf-8').read().split('\n')
    for item in data:
        processed_item = item.lower()
        for s_t in stop_words:

            processed_item = re.sub(rf" {s_t} ", ' ', processed_item)

        processed_item = re.sub(f'[{punctuation}]',' ',processed_item)
        processed_item = re.sub(f'\s+',' ',processed_item)
        temp = underthesea.sent_tokenize(processed_item)
        sentences += temp

    tokenize_data = [underthesea.word_tokenize(i) for i in sentences]

    return tokenize_data, sentences

In [11]:
token1,sentences1 = process_data(final_dat1.Text.values)
token2,sentences2 = process_data(final_dat2.Text.values)
token3,sentences3 = process_data(final_dat3.Text.values)

In [24]:
sentences1

['rộng rãi ks vắng dịch vụ chất thiếu',
 'địa thuận tiện vòng kính 1 5km quán ngon',
 'phục vụ view đẹp vị trí',
 'thuận tiện sạch vui vẻ hài lòng',
 'vị trí đẹp quán bar view đẹp nhân viên thân thiện',
 ' co view huong ho tay sach se nhan vien tan tinh',
 'phòng ốc sạch giường thoải mái nhân viên thân thiện',
 'gần hồ tây view hồ lãng mạn đi dọc hồ tây món hà nội phòng sạch đẹp tiện nghi view hồ tây tuyệt phí thuê xe ô tô sạn đi yên tử chùa hương giá cáo nhiều',
 'hình thức thể quyết định nội dung h atildey tận hưởng cảm nhận với tôi tuyệt vời',
 'tôi 4 phòng phòng bụi tối đi ngủ đi chăn drap kiến vàng nhỏ tầng 9 vật dụng phòng tắm trà cafe thiếu nhân viên lễ tân nhiệt tình chịu',
 'tôi 2 định nữa nhiên check in chẳng chẳng quan tâm 1 khiêng hành lý phòng hic hành lý xe phòng nhân viên thương',
 'tôi sạn 1 phòng premium khuyến m atildei chudu24 giá khuyến m atildei phòng ngỡ ngàng sạch đẹp view đẹp thái độ phục vụ nhân viên tốt chắc chắn dịch vụ sạn nay',
 'tôi hài sạn đội ngũ nhân vi

In [12]:
token1

[['rộng rãi', 'ks', 'vắng', 'dịch vụ', 'chất', 'thiếu'],
 ['địa thuận tiện', 'vòng', 'kính', '1', '5', 'km', 'quán', 'ngon'],
 ['phục vụ', 'view', 'đẹp', 'vị trí'],
 ['thuận tiện', 'sạch', 'vui vẻ', 'hài lòng'],
 ['vị trí', 'đẹp', 'quán', 'bar', 'view', 'đẹp', 'nhân viên', 'thân thiện'],
 ['co', 'view', 'huong', 'ho', 'tay', 'sach', 'se', 'nhan vien', 'tan tinh'],
 ['phòng ốc', 'sạch', 'giường', 'thoải mái', 'nhân viên', 'thân thiện'],
 ['gần',
  'hồ',
  'tây',
  'view',
  'hồ',
  'lãng mạn',
  'đi',
  'dọc',
  'hồ',
  'tây',
  'món',
  'hà nội',
  'phòng',
  'sạch',
  'đẹp',
  'tiện nghi',
  'view',
  'hồ tây',
  'tuyệt phí',
  'thuê',
  'xe',
  'ô tô sạn',
  'đi',
  'yên tử',
  'chùa',
  'hương',
  'giá cáo',
  'nhiều'],
 ['hình thức thể',
  'quyết định',
  'nội dung',
  'h',
  'atildey',
  'tận hưởng',
  'cảm nhận',
  'với',
  'tôi',
  'tuyệt vời'],
 ['tôi',
  '4',
  'phòng',
  'phòng',
  'bụi',
  'tối',
  'đi',
  'ngủ',
  'đi',
  'chăn',
  'drap kiến',
  'vàng',
  'nhỏ',
  'tầng',


In [46]:
# clean_dat1

,Text,Label
0,Rộng rãi KS vắng Các dịch vụ chất,0
1,Địa thuận tiện vòng kính 1 5 km quán ngon,0
2,Phục vụ view đẹp vị trí,1
3,thuận tiện sạch vui vẻ hài lòng,1
4,Vị trí đẹp Có quán bar view đẹp Nhân viên thân...,1
...,...,...
2995,Đối diện thung lũng Một kỳ nghỉ tuyệt vời Sapa...,0
2996,Cảm Sapa View Phòng ốc đẹp sạch Nhân viên phục...,1
2997,Xin chào Chudu24 Xin cám ơn dịch vụ booking Ch...,0
2998,Đầu tiên ưu Phòng sạch nội thất view đẹp Nhược...,-1


In [10]:
token3

[['ga', 'giường', 'sạch', 'nhân viên', 'quên', 'dọn', 'phòng', 'một', 'ngày'],
 ['nv',
  'nhiệt tình',
  'phòng',
  'sạch sẽ',
  'tiện nghi',
  'thuận tiện',
  'di địa',
  'phòng',
  'gián'],
 ['đi',
  'biển',
  'gần',
  'tiện',
  'đi',
  'phòng',
  'view',
  'biển',
  'cửa sổ',
  'view',
  'biển',
  'bé'],
 ['tất',
  'sạch sẽ',
  'giường',
  'ngủ',
  'thoải mái',
  'quạt',
  'điện',
  'hòa chút',
  'bất tiện'],
 ['giường',
  'sạch sẽ',
  'thoải mái',
  'thiết thiết trang',
  'sẵn',
  'lười',
  'phòng',
  'tắm',
  'chia',
  'ko',
  'gian',
  'khô',
  'ướt'],
 ['phòng ốc',
  'mới',
  'trọng giường',
  'thoải mái',
  'sạch',
  'chỗ',
  'xe hơi',
  'chật chọi'],
 ['khăn', 'tắm', 'cũ', 'dơ đồ', 'toilet đồ', 'lại', 'đồ nguyên', 'seal'],
 ['phòng',
  'đẹp',
  'sạch',
  'biển',
  'nhân viên',
  'thương',
  'phòng',
  'view',
  'biển',
  'tv cáp'],
 ['rộng rãi', 'sạch sẽ', 'chỗ', 'phòng', 'bắt', 'wifi'],
 ['khách sạn',
  'đẹp trọng',
  'nhân viên',
  'vui vẻ',
  'nhiệt tình',
  'giá',
  'hợp l

In [13]:
def process_data2(data):
    data_doc = []
    stop_words = open('Dataset_For_Work\\vietnamese-stopwords.txt','r',encoding='utf-8').read().split('\n')
    for item in data:
        processed_item = item.lower()
        for s_t in stop_words:
            processed_item = re.sub(rf" {s_t} ", ' ', processed_item)
        processed_item = re.sub(f'[{punctuation}]',' ',processed_item)
        processed_item = re.sub(f'\s+',' ',processed_item)
        temp = underthesea.sent_tokenize(processed_item)
        tokenize_data = []
        for i in temp:
          tokenize_data += underthesea.word_tokenize(i)
        data_doc.append(tokenize_data)
    return data_doc

In [14]:
process_dat1 = process_data2(final_dat1.Text.values)
process_dat2 = process_data2(final_dat2.Text.values)
process_dat3 = process_data2(final_dat3.Text.values)

In [26]:
process_dat1

[['rộng rãi', 'ks', 'vắng', 'dịch vụ', 'chất', 'thiếu'],
 ['địa thuận tiện', 'vòng', 'kính', '1', '5', 'km', 'quán', 'ngon'],
 ['phục vụ', 'view', 'đẹp', 'vị trí'],
 ['thuận tiện', 'sạch', 'vui vẻ', 'hài lòng'],
 ['vị trí', 'đẹp', 'quán', 'bar', 'view', 'đẹp', 'nhân viên', 'thân thiện'],
 ['co', 'view', 'huong', 'ho', 'tay', 'sach', 'se', 'nhan vien', 'tan tinh'],
 ['phòng ốc', 'sạch', 'giường', 'thoải mái', 'nhân viên', 'thân thiện'],
 ['gần',
  'hồ',
  'tây',
  'view',
  'hồ',
  'lãng mạn',
  'đi',
  'dọc',
  'hồ',
  'tây',
  'món',
  'hà nội',
  'phòng',
  'sạch',
  'đẹp',
  'tiện nghi',
  'view',
  'hồ tây',
  'tuyệt phí',
  'thuê',
  'xe',
  'ô tô sạn',
  'đi',
  'yên tử',
  'chùa',
  'hương',
  'giá cáo',
  'nhiều'],
 ['hình thức thể',
  'quyết định',
  'nội dung',
  'h',
  'atildey',
  'tận hưởng',
  'cảm nhận',
  'với',
  'tôi',
  'tuyệt vời'],
 ['tôi',
  '4',
  'phòng',
  'phòng',
  'bụi',
  'tối',
  'đi',
  'ngủ',
  'đi',
  'chăn',
  'drap kiến',
  'vàng',
  'nhỏ',
  'tầng',


In [22]:
def convert_data_to_vector(data,model,max_length):

    vector_data = []
    for item in data:
        temp = []
        print(item)
        for word in item:
            try:
                temp.append(model.wv.get_vector(word))
            except KeyError:
                print(word)
                print('Từ này không có trong từ điển :))')
                continue
        
        # Padding câu để đồng nhất kích thước của vector, phục vụ cho quá trình train
        while len(temp) < max_length:
            temp.append(np.zeros(model.wv.vector_size))  # Thêm vector 0
        
        # Cắt bớt nếu câu dài hơn max_length
        temp = temp[:max_length]
        
        vector_data.append(np.array(temp))
    
    # Chuyển embedding vector sang Tensor
    vector_data = np.array(vector_data)  # (batch_size, max_length, embedding_dim)
    return torch.tensor(vector_data, dtype=torch.float32)


#Tensor chứa list các vector câu 
#Kích thước cảu Tensor là (số lượng câu,kích thước của 1 câu,số chiều của vector embedding)
#Ví dụ có câu #Tôi đi học
#Tensor có dạng
#[[vector tôi],[vector đi],[vector học]]

In [16]:
MAX_LENGTH = 100

In [17]:
model = gensim.models.Word2Vec(sentences=token1 + token2 + token3,
                               min_count=1,
                               vector_size=128,
                               alpha=0.001,
                               window=5,
                               epochs=2000,
                               )

In [18]:
from torch.utils.data import DataLoader, TensorDataset

In [27]:
# Chuyển đổi x_train và x_test sang vector rồi padding
x_train_vector = convert_data_to_vector(data=process_dat1, model=model, max_length=MAX_LENGTH)
x_test_vector = convert_data_to_vector(data=process_dat2, model=model, max_length=MAX_LENGTH)


#Chuyển nhãn từ -1 -> 0 , 0->1 , 1->2 để dùng softmax
y_train = np.array(final_dat1.Label.values, dtype=np.int64)+1
y_test = np.array(final_dat2.Label.values, dtype=np.int64)+1

y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(x_train_vector, y_train)
test_dataset = TensorDataset(x_test_vector, y_test)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

#Batch size là số lượng mẫu dữ liệu được xử lý cùng lúc trong 1 lần huấn luyện
#Ví dụ 300 mẫu dữ liệu thì nó sẽ tác động 128 mẫu đầu / batch
#rồi tiếp tục cho đến khi áp dụng phần dư còn lại nốt tính là 1 batch nữa

['rộng rãi', 'ks', 'vắng', 'dịch vụ', 'chất', 'thiếu']
['địa thuận tiện', 'vòng', 'kính', '1', '5', 'km', 'quán', 'ngon']
['phục vụ', 'view', 'đẹp', 'vị trí']
['thuận tiện', 'sạch', 'vui vẻ', 'hài lòng']
['vị trí', 'đẹp', 'quán', 'bar', 'view', 'đẹp', 'nhân viên', 'thân thiện']
['co', 'view', 'huong', 'ho', 'tay', 'sach', 'se', 'nhan vien', 'tan tinh']
['phòng ốc', 'sạch', 'giường', 'thoải mái', 'nhân viên', 'thân thiện']
['gần', 'hồ', 'tây', 'view', 'hồ', 'lãng mạn', 'đi', 'dọc', 'hồ', 'tây', 'món', 'hà nội', 'phòng', 'sạch', 'đẹp', 'tiện nghi', 'view', 'hồ tây', 'tuyệt phí', 'thuê', 'xe', 'ô tô sạn', 'đi', 'yên tử', 'chùa', 'hương', 'giá cáo', 'nhiều']
['hình thức thể', 'quyết định', 'nội dung', 'h', 'atildey', 'tận hưởng', 'cảm nhận', 'với', 'tôi', 'tuyệt vời']
['tôi', '4', 'phòng', 'phòng', 'bụi', 'tối', 'đi', 'ngủ', 'đi', 'chăn', 'drap kiến', 'vàng', 'nhỏ', 'tầng', '9', 'vật dụng', 'phòng', 'tắm', 'trà', 'cafe', 'thiếu', 'nhân viên', 'lễ tân', 'nhiệt tình', 'chịu']
['tôi', '2', 'đ

In [28]:
model.wv.most_similar('thân thiện')

[('phục vụ', 0.6845700740814209),
 ('chu đáo', 0.6630825996398926),
 ('nhiệt tình', 0.6581295728683472),
 ('tốt', 0.6499554514884949),
 ('vui vẻ', 0.6448497772216797),
 ('niềm nở', 0.6196531653404236),
 ('sạch sẽ', 0.617756187915802),
 ('bữa', 0.6143447160720825),
 ('tiện nghi', 0.6002974510192871),
 ('thoải mái', 0.583195686340332)]

In [29]:
token1

[['rộng rãi', 'ks', 'vắng', 'dịch vụ', 'chất', 'thiếu'],
 ['địa thuận tiện', 'vòng', 'kính', '1', '5', 'km', 'quán', 'ngon'],
 ['phục vụ', 'view', 'đẹp', 'vị trí'],
 ['thuận tiện', 'sạch', 'vui vẻ', 'hài lòng'],
 ['vị trí', 'đẹp', 'quán', 'bar', 'view', 'đẹp', 'nhân viên', 'thân thiện'],
 ['co', 'view', 'huong', 'ho', 'tay', 'sach', 'se', 'nhan vien', 'tan tinh'],
 ['phòng ốc', 'sạch', 'giường', 'thoải mái', 'nhân viên', 'thân thiện'],
 ['gần',
  'hồ',
  'tây',
  'view',
  'hồ',
  'lãng mạn',
  'đi',
  'dọc',
  'hồ',
  'tây',
  'món',
  'hà nội',
  'phòng',
  'sạch',
  'đẹp',
  'tiện nghi',
  'view',
  'hồ tây',
  'tuyệt phí',
  'thuê',
  'xe',
  'ô tô sạn',
  'đi',
  'yên tử',
  'chùa',
  'hương',
  'giá cáo',
  'nhiều'],
 ['hình thức thể',
  'quyết định',
  'nội dung',
  'h',
  'atildey',
  'tận hưởng',
  'cảm nhận',
  'với',
  'tôi',
  'tuyệt vời'],
 ['tôi',
  '4',
  'phòng',
  'phòng',
  'bụi',
  'tối',
  'đi',
  'ngủ',
  'đi',
  'chăn',
  'drap kiến',
  'vàng',
  'nhỏ',
  'tầng',


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SentimentCNNLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1, num_filters=128, filter_sizes=[2, 3, 4], num_classes=3, dropout_rate=0.5):
        super(SentimentCNNLSTM, self).__init__()
        
        # Convolutional layers với BatchNorm và ReLU
        self.convs = nn.ModuleList([
            nn.Sequential(
                nn.Conv1d(in_channels=input_dim, 
                          out_channels=num_filters, 
                          kernel_size=fs),
                nn.BatchNorm1d(num_filters),
                nn.ReLU()
            )
            for fs in filter_sizes
        ])
        
        # LSTM layer
        self.lstm = nn.LSTM(input_size=len(filter_sizes) * num_filters, 
                            hidden_size=hidden_dim, 
                            num_layers=num_layers, 
                            batch_first=True, 
                            bidirectional=True)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout_rate)
        
        # Fully connected layer
        self.fc = nn.Linear(2 * hidden_dim, num_classes)  # Bidirectional nên 2 * hidden_dim

    def forward(self, x):
        # Đầu vào: (batch_size, sequence_length, embedding_dim)
        # Đổi chiều thành: (batch_size, embedding_dim, sequence_length)
        x = x.permute(0, 2, 1)
        
        # Áp dụng convolution và global max pooling
        conv_results = []
        for conv in self.convs:
            conved = conv(x)  # (batch_size, num_filters, new_seq_length)
            pooled = F.max_pool1d(conved, kernel_size=conved.shape[2]).squeeze(2)  # (batch_size, num_filters)
            conv_results.append(pooled)
        
        # Nối kết quả từ các filters lại thành 1 tensor
        x = torch.cat(conv_results, dim=1)  # (batch_size, len(filter_sizes) * num_filters)
        
        # Thêm chiều sequence_length=1 để phù hợp đầu vào LSTM
        x = x.unsqueeze(1)  # (batch_size, 1, len(filter_sizes) * num_filters)
        
        # LSTM layer
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch_size, seq_length=1, 2*hidden_dim)
        lstm_out = lstm_out[:, -1, :]  # Lấy output cuối cùng (batch_size, 2*hidden_dim)
        
        # Dropout
        x = self.dropout(lstm_out)
        
        # Fully connected layer
        logits = self.fc(x)
        
        return logits


# Khởi tạo mô hình
input_dim = x_train_vector.shape[2]  # Số chiều embedding (embedding_dim)
hidden_dim = 128  # Kích thước hidden state của LSTM
num_layers = 1  # Số layer LSTM
num_filters = 128  # Số lượng filters cho CNN
filter_sizes = [2, 3, 4]  # Kích thước các filters
num_classes = 3  # Số lớp đầu ra
dropout_rate = 0.5  # Tỉ lệ dropout

# Khởi tạo mô hình SentimentCNNLSTM
model = SentimentCNNLSTM(
    input_dim=input_dim, 
    hidden_dim=hidden_dim, 
    num_layers=num_layers,
    num_filters=num_filters, 
    filter_sizes=filter_sizes, 
    num_classes=num_classes,
    dropout_rate=dropout_rate
)

# Chọn hàm loss và optimizer
criterion = nn.CrossEntropyLoss()  # CrossEntropyLoss cho bài toán phân loại nhiều lớp
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)  # Adam optimizer với weight decay


In [64]:
from tqdm.autonotebook import tqdm

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Tham số huấn luyện
num_epochs = 1000

# Vòng lặp huấn luyện
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()  # Reset gradient
        outputs = model(x_batch)  # Forward pass
        loss = criterion(outputs, y_batch)  # Tính loss
        loss.backward()  # Backward pass
        optimizer.step()  # Cập nhật trọng số
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")

# Đánh giá mô hình trên tập test
model.eval()  # Chuyển mô hình sang chế độ đánh giá
y_true = []
y_pred = []

with torch.no_grad():  # Không tính gradient khi test
    for x_batch, y_batch in test_loader:
        outputs = model(x_batch)  # Forward pass
        _, predicted = torch.max(outputs, 1)  # Lấy chỉ số của lớp có xác suất cao nhất
        
        y_true.extend(y_batch.cpu().numpy())  # Ghi lại nhãn thực tế
        y_pred.extend(predicted.cpu().numpy())  # Ghi lại nhãn dự đoán

# Tính Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

# Tính Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Hiển thị Confusion Matrix bằng Seaborn
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Neutral', 'Positive'], yticklabels=['Negative', 'Neutral', 'Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


Epoch [1/1000], Loss: 0.2256
Epoch [2/1000], Loss: 0.1698
Epoch [3/1000], Loss: 0.1181
Epoch [4/1000], Loss: 0.0803
Epoch [5/1000], Loss: 0.0593
Epoch [6/1000], Loss: 0.0411
Epoch [7/1000], Loss: 0.0279
Epoch [8/1000], Loss: 0.0306
Epoch [9/1000], Loss: 0.0210
Epoch [10/1000], Loss: 0.0167
Epoch [11/1000], Loss: 0.0119
Epoch [12/1000], Loss: 0.0107
Epoch [13/1000], Loss: 0.0227
Epoch [14/1000], Loss: 0.0122
Epoch [15/1000], Loss: 0.0085
Epoch [16/1000], Loss: 0.0099
Epoch [17/1000], Loss: 0.0064
Epoch [18/1000], Loss: 0.0067
Epoch [19/1000], Loss: 0.0083
Epoch [20/1000], Loss: 0.0045
Epoch [21/1000], Loss: 0.0032
Epoch [22/1000], Loss: 0.0026
Epoch [23/1000], Loss: 0.0021
Epoch [24/1000], Loss: 0.0020
Epoch [25/1000], Loss: 0.0017
Epoch [26/1000], Loss: 0.0015
Epoch [27/1000], Loss: 0.0015
Epoch [28/1000], Loss: 0.0013
Epoch [29/1000], Loss: 0.0013
Epoch [30/1000], Loss: 0.0012
Epoch [31/1000], Loss: 0.0011
Epoch [32/1000], Loss: 0.0011
Epoch [33/1000], Loss: 0.0011
Epoch [34/1000], Lo